In [1]:
import pandas as pd
import requests
import aiohttp
import asyncio
import json
import os
import sys
from aiohttp import ClientSession
from urllib.error import HTTPError

In [2]:
API = "https://api.elections.org.za"
token = ''
headers = {"Authorization": "Bearer " + token}

In [3]:
Councillors = []

In [4]:
wards_df = pd.read_csv('Wards.csv')
Wards = wards_df.values.tolist()

In [5]:
async def get_data(endpoint, query, session):
    url = API + endpoint + str(query)
    try:
        response = await session.request(method='GET', url=url, headers=headers)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error ocurred: {err}")
    response_json = await response.json()
    return response_json

In [6]:
async def run_program(endpoint, query, session):
    try:
        response = await get_data(endpoint, query, session)
        print(str(response['Delimitation']['ProvinceID']) + ' - ' + str(response['Delimitation']['MunicipalityID']) + ' - ' + str(response['Delimitation']['WardID']), end='\r')
        sys.stdout.flush()
        Councillors.append(
            {
                'Name': response['Name'],
                'ProvinceID': response['Delimitation']['ProvinceID'],
                'MunicipalityID': response['Delimitation']['MunicipalityID'],
                'WardID': response['Delimitation']['WardID'],
                'VDNumber': response['Delimitation']['VDNumber'],
                'PartyID': response['PartyDetail']['ID'],
                'Party': response['PartyDetail']['Name'],
                'PartyAbbreviation': response['PartyDetail']['Abbreviation'],
                'ContactPerson': response['Municipality']['ContactDetails']['ContactPerson'],
                'Tel': response['Municipality']['ContactDetails']['Tel'],
                'Fax': response['Municipality']['ContactDetails']['Fax'],
                'PostalAddress': response['Municipality']['ContactDetails']['PostalAddress'],
                'WebsiteUrl': response['Municipality']['ContactDetails']['WebsiteUrl']
            }
        )
    except Exception as err:
        print(f"Exception occured: {err}")
        pass

In [7]:
async with ClientSession() as session:
    await asyncio.gather(*[run_program('/api/LGEWardCouncilor/WardID/', ward[2], session) for ward in Wards])
    sys.stdout.flush()
    print('Saving to CSV...')
    councillors_df = pd.DataFrame(Councillors) 
    councillors_df.to_csv('Councillors.csv', index=False)
    print('Done')

Saving to CSV...007
Done
